In [152]:
import random
import io
import numpy as np
import heapq
import json
import operator
import pandas as pd
from collections import Counter
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from stop_words import get_stop_words
stop_words = get_stop_words('english')

from gensim.models.doc2vec import LabeledSentence, Doc2Vec

import multiprocessing

cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1, "this will be painfully slow otherwise"

In [26]:
path_to_data = '../data/'

##########################
# load some of the files #                           
##########################

training = pd.read_csv(path_to_data + 'training_set.csv', sep=',', header=0)
training_info = pd.read_csv(path_to_data + 'training_info.csv', sep=',', header=0)
test = pd.read_csv(path_to_data + 'test_set.csv', sep=',', header=0)
test_info = pd.read_csv(path_to_data + 'test_info.csv', sep=',', header=0)

In [ ]:
# Correct dates and put datetime format
# We do that because we noticed test_set is only composed of email posterior to the ones of train_set. 
# Datetime format allows to simulate posteriority in our train/test split
from datetime import datetime

for row in training_info.sort(['date']).iterrows():
    date = row[1]['date']
    if date[:3] == '000':
        date = '2' + date[1:]
        
    training_info.loc[row[0], 'date'] = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')

## Text Embeddings

#### Doc2vec

function "most_similar" from doc2vec doesn't work. Impossible to get closest documents. Maybe the corpus isn't big enough to train a neural net. Trying to recover closest docs with cosine similarity doesn't seem to work either.

In [45]:
# In this cell we prepare labels for emails in case we want to perform a multilabel classification

#To see the code that allowed to create that list, see "archive"
with io.open('../data/person_id.txt') as json_data:
    person_id = json.load(json_data)

#Create labels for emails
labels = []
for row in training_info.iterrows():
    recipients_id = []
    for recipients in row[1]['recipients'].split():
        if '@' in recipients:
            #print recipients
            recipients_id.append(person_id[recipients])
            
    labels.append(recipients_id)
documents = training_info['body'].values

from sklearn.preprocessing import MultiLabelBinarizer
labels_binary = MultiLabelBinarizer().fit_transform(labels)

In [110]:
#Create a list of documents that fit Doc2Vec expected format. For some reason document have to receive a 'label', 
#but they are more like tags, don't know if it could be used for classification purposes. 
documents = []

for row in training_info.iterrows():
    document = LabeledSentence(words=row[1]['body'].split(), tags=['SENTENCE_'+str(row[0])]) 
    documents.append(document)

In [114]:
model = Doc2Vec(dm=1, dm_concat=1, size=100, window=5, negative=5, hs=0, min_count=2, workers=cores)
model.build_vocab(documents)

In [120]:
for epoch in range(10):
    random.shuffle(documents)
    model.train(documents)
    print 'done for epoch: ', str(epoch) 

done for epoch:  0
done for epoch:  1
done for epoch:  2
done for epoch:  3
done for epoch:  4
done for epoch:  5
done for epoch:  6
done for epoch:  7
done for epoch:  8
done for epoch:  9


In [ ]:
# save model
model.save('../data/enron.d2v')

### TF-IDF

In [4]:
tfidf = TfidfVectorizer(stop_words = stop_words)
#NOTE: we use toarray() for the moment because easier to handle, but less efficient, see later if better is possible
array_embedding_sparse = tfidf.fit_transform(training_info['body'].values)

## First Algorithm

The idea here is to embed the email body, find its sender, find the 30 closest emails in the embedding space, which were written by the sender, construct a dictionnary of recipients those 30 emails were addressed to, and pick the 10 most frequent reciepient

In [11]:
def most_n_similar_sklearn(array_embedding_sparse, query_id):
    
    similarities = cosine_similarity(array_embedding_sparse, array_embedding_sparse[query_id])
    if int(round(sorted(similarities[:,0], reverse=True)[0])) ==1:
        closest_ids = similarities[:,0].argsort()[::-1][1:]
    else:
        closest_ids = similarities[:,0].argsort()[::-1]
    
    return closest_ids

In [77]:
def get_sender(query_mid, training):
    for row in training.iterrows():
        mids = row[1]['mids'].split()
        for mid in mids:
            if int(mid) == query_mid:
                sender = row[1]['sender']
                break
    return sender

def get_n_closest_emails(sender, n, closest_ids, training, training_info):
    # Get all emails' mids from query sender
    all_emails_from_sender_mids = [int(k) for k in training[training['sender']==sender]['mids'].values[0].split()]

    # Get emails' index from query sender
    all_emails_from_sender_ids = training_info[training_info['mid'].isin(all_emails_from_sender_mids)].index.values

    # Get the closest emails WRITTEN BY THE SENDER
    closest_ids_per_sender = []
    for idx in closest_ids:
        if idx in all_emails_from_sender_ids:
            closest_ids_per_sender.append(idx)
        if len(closest_ids_per_sender) == n:
            break
            
    return closest_ids_per_sender

def get_10_recipients(closest_ids_per_sender, training_info):
    dic_of_recipients = {}
    for idx in closest_ids_per_sender:
        recipients = training_info.loc[idx,'recipients'].split()
        for recipient in recipients:
            if '@' in recipient:
                if recipient not in dic_of_recipients.keys():
                    dic_of_recipients[recipient] = 1
                else:
                    dic_of_recipients[recipient] += 1

    suggested_10_recipients = heapq.nlargest(10, dic_of_recipients, key=dic_of_recipients.get)
    
    return suggested_10_recipients

def mean_ap(suggested_10_recipients, ground_truth):
    MAP = 0
    correct_guess = 0
    for i, suggestion in enumerate(suggested_10_recipients):
        if suggestion in ground_truth:
            correct_guess +=1
            MAP += float(correct_guess)/(i+1)
    MAP = float(MAP)/min(10, len(ground_truth))
    return MAP

In [102]:
query_id = 6
all_mean_ap = []

for query_id in training_info.head(1000).index.values:
    
    if query_id%100==0:
        print query_id
    # number of closest neighbors to collect recipients from:
    n = 30
    ground_truth = training_info['recipients'][query_id].split()

    closest_ids = most_n_similar_sklearn(array_embedding_sparse, query_id)
    query_mid = training_info['mid'][query_id]

    # find the sender from the query email
    sender = get_sender(query_mid, training)

    # find the closest emails to the query one, written by the sender
    closest_ids_per_sender = get_n_closest_emails(sender, n, closest_ids, training, training_info)

    # Create dictionnary of all recipient for the 30 most similar emails, and get frequency
    # For the moment it is only the brute frequency, maybe we could refine this by adding wheights according to the closseness of the email
    suggested_10_recipients = get_10_recipients(closest_ids_per_sender, training_info)

    # print
    # print suggested_10_recipients
    all_mean_ap.append(mean_ap(suggested_10_recipients, ground_truth))

['portland.desk@enron.com']

['portland.desk@enron.com', 'bill.williams@enron.com', 'chris.stokley@enron.com', 'kathy.axford@enron.com', 'holden.salisbury@enron.com', 'kate.symes@enron.com', 'm..driscoll@enron.com', 'c..bland@enron.com', 'f..calger@enron.com', 'diana.scholtes@enron.com']
1.0


## Test the algorithm

In [148]:
X_test = training_info[training_info.date > datetime(2001, 10, 15)]
X_train = training_info[training_info.date <= datetime(2001, 10, 15)]

In [162]:
X_test.head(10)

,mid,date,body,recipients
31,1189,2001-11-19 08:16:20,The following expense report is ready for appr...,kevin.hyatt@enron.com
40,2295,2001-10-23 14:37:36,State Regulations Forces Questar to Cancel Par...,mark.mcconnell@enron.com martin.gonzalez@enron...
118,5219,2001-10-15 08:58:07,Attached is a NON BINDING letter of interest t...,james.centilli@enron.com mansoor.abdmoulaie@en...
119,5480,2001-10-29 12:20:59,Tracy: Since I believe that Rod will also b...,tracy.geaccone@enron.com rod.hayslett@enron.com
120,5536,2001-10-18 07:17:26,"Good morning, Tracy! Stan wants us to look a...",tracy.geaccone@enron.com
133,7014,2001-10-22 16:10:01,Please look at the attached. If there are som...,stephen.dowd@enron.com bob.chandler@enron.com ...
134,7422,2001-10-29 11:33:28,Richard: I understand from Elliot that we want...,richard.ring@enron.com elliot.mainzer@enron.co...
135,7476,2001-10-29 11:33:28,Richard: I understand from Elliot that we want...,richard.ring@enron.com elliot.mainzer@enron.co...
142,7722,2001-10-15 11:18:38,"We are hoping to have the retreat on Thursday,...",jeff.shields@enron.com jesse.bryson@enron.com ...
143,7723,2001-10-16 10:57:44,Richard: I suggest you get in on the evening o...,richard.ring@enron.com


In [ ]:
tfidf = TfidfVectorizer(stop_words = stop_words)
array_embedding_sparse = tfidf.fit_transform(X_train['body'].values)

In [165]:
all_mean_ap = []

#perform tf-idf on X_train

count = 0
for query_id in X_test.head(2000).index.values:
    count+=1
    if count%100==0:
        print count
    # number of closest neighbors to collect recipients from:
    n = 30
    ground_truth = training_info['recipients'][query_id].split()

    closest_ids = most_n_similar_sklearn(array_embedding_sparse, query_id)
    query_mid = X_test['mid'][query_id]

    # find the sender from the query email
    sender = get_sender(query_mid, training)

    # find the closest emails (written by the sender) to the query one
    closest_ids_per_sender = get_n_closest_emails(sender, n, closest_ids, training, X_train)

    # Create dictionnary of all recipient for the 30 most similar emails, and get frequency
    # For the moment it is only the brute frequency, maybe we could refine this by adding wheights according to the closseness of the email
    suggested_10_recipients = get_10_recipients(closest_ids_per_sender, X_train)

    #print ground_truth
    #print
    #print suggested_10_recipients
    all_mean_ap.append(mean_ap(suggested_10_recipients, ground_truth))

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000


In [166]:
np.mean(all_mean_ap)

0.22607515384857646

## Baseline

In [158]:
################################
# create some handy structures #                    
################################
                            
# convert training set to dictionary
emails_ids_per_sender = {}
for index, series in training.iterrows():
    row = series.tolist()
    sender = row[0]
    ids = row[1:][0].split(' ')
    emails_ids_per_sender[sender] = ids

# save all unique sender names
all_senders = emails_ids_per_sender.keys()

In [159]:
# create address book with frequency information for each user
address_books = {}
i = 0

for sender, ids in emails_ids_per_sender.iteritems():
    recs_temp = []
    for my_id in ids:
        recipients = training_info[training_info['mid']==int(my_id)]['recipients'].tolist()
        recipients = recipients[0].split(' ')
        # keep only legitimate email addresses
        recipients = [rec for rec in recipients if '@' in rec]
        recs_temp.append(recipients)
    # flatten    
    recs_temp = [elt for sublist in recs_temp for elt in sublist]
    # compute recipient counts
    rec_occ = dict(Counter(recs_temp))
    # order by frequency
    sorted_rec_occ = sorted(rec_occ.items(), key=operator.itemgetter(1), reverse = True)
    # save
    address_books[sender] = sorted_rec_occ
    
    if i % 10 == 0:
        print i
    i += 1
    

0
10
20
30
40
50
60
70
80
90
100
110
120


In [160]:
#save all unique recipient names    
all_recs = list(set([elt[0] for sublist in address_books.values() for elt in sublist]))

# save all unique user names 
all_users = []
all_users.extend(all_senders)
all_users.extend(all_recs)
all_users = list(set(all_users))

In [161]:
#############
# baselines #                           
#############

# will contain email ids, predictions for random baseline, and predictions for frequency baseline
predictions_per_sender = {}

# number of recipients to predict
k = 10

for index, row in test.iterrows():
    name_ids = row.tolist()
    sender = name_ids[0]
    # get IDs of the emails for which recipient prediction is needed
    ids_predict = name_ids[1].split(' ')
    ids_predict = [int(my_id) for my_id in ids_predict]
    random_preds = []
    freq_preds = []
    # select k most frequent recipients for the user
    k_most = [elt[0] for elt in address_books[sender][:k]]
    for id_predict in ids_predict:
        # select k users at random
        random_preds.append(random.sample(all_users, k))
        # for the frequency baseline, the predictions are always the same
        freq_preds.append(k_most)
    predictions_per_sender[sender] = [ids_predict,random_preds,freq_preds]	

In [162]:
#################################################
# write predictions in proper format for Kaggle #                           
#################################################

path_to_results = '../submission/'

with open(path_to_results + 'predictions_random.csv', 'wb') as my_file:
    my_file.write('mid,recipients' + '\n')
    for sender, preds in predictions_per_sender.iteritems():
        ids = preds[0]
        random_preds = preds[1]
        for index, my_preds in enumerate(random_preds):
            my_file.write(str(ids[index]) + ',' + ' '.join(my_preds) + '\n')

with open(path_to_results + 'predictions_frequency.csv', 'wb') as my_file:
    my_file.write('mid,recipients' + '\n')
    for sender, preds in predictions_per_sender.iteritems():
        ids = preds[0]
        freq_preds = preds[2]
        for index, my_preds in enumerate(freq_preds):
            my_file.write(str(ids[index]) + ',' + ' '.join(my_preds) + '\n')